In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"]= (10,2)
from statsmodels.tsa.stattools import adfuller
import warnings
warnings.filterwarnings("ignore")
import plotly.express as px

In [2]:
def sliderPlot(df1,t=None):
    if t =="line":
        fig = px.line(x=df1.index, y=df1)
    else:
        fig = px.scatter(x=df1.index, y=df1)
    fig.update_xaxes(
        rangeslider_visible=True,
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(count=2, label="2y", step="year", stepmode="backward"),
                dict(count=3, label="3y", step="year", stepmode="backward"),
                dict(step="all")
            ])))
    fig.show()
    return
def scatter(df1):
    plt.scatter(df1.index, df1, color='k',s=10)
    plt.xlabel('timestamp')
    # plt.title('current')
    plt.show()

In [31]:
lst = ['wm',"ac1",'ac2',"f",'tv']
path = "d:/NILM/Data_files/Parquet/Appliances12_06-04_08/"
l = []
for i in range(len(lst)):
    df = pd.read_parquet(f"{path}{lst[i]}.parquet")
    # print(f"no of records in {lst[i]}:",len(df))
    x = len(df)
    df.drop_duplicates(inplace=True)
    df.set_index(['timestamp'],inplace=True ,drop=True)
    # print(f"null in {lst[i]}:",df.isna().sum().sum())
    # print(f"duplicates in {lst[i]}:",df.duplicated().sum())
    df.drop(df[(df['frequency']>51) | (df['frequency']<49)].index, inplace=True)
    df.drop(df[(df['PF']>1) | (df['PF']<0)].index, inplace=True)
    print(f"no. of dropped records in {lst[i]} :",(x - len(df)))    
    print(f"no of records in {lst[i]}:",len(df))
    l.append(df)


no. of dropped records in wm : 0
no of records in wm: 20885
no. of dropped records in ac1 : 9
no of records in ac1: 337732
no. of dropped records in ac2 : 39
no of records in ac2: 426542
no. of dropped records in f : 220
no of records in f: 4043992
no. of dropped records in tv : 15
no of records in tv: 414699


washing machine


In [32]:
df = l[0]
dit = {}
lst = []
x = 0
df.reset_index(inplace=True)
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index(['timestamp'],inplace=True ,drop=True)
df['power'] = (df['voltage'] * df['current'] * df['PF']).round(2)
unique_dates = pd.Series(df.index.date).unique()
for date in unique_dates:
    lst2 = []
    result = df[df.index.to_series().dt.date == date]
    hours = result.index.hour.unique()
    for i in hours:
        daate = f"{date} {i:02d}"
        result_hour = result.loc[daate]
        data = result_hour[['power',"voltage",'current']].resample(rule="1s").mean()
        print(daate,len(data))
        # y = data['current'].isna().sum().sum()
        lst.append(data)
        lst2.append(i)
        x+=1
    dit[f"{date}"]= lst2
print("days when appliance is active :", len(unique_dates))
print("no. of hours:", x)                

                

2013-06-14 01 1939
2013-06-17 04 1507
2013-06-21 00 4
2013-06-21 01 1674
2013-06-24 05 1330
2013-06-28 01 678
2013-06-28 02 1538
2013-07-02 05 1677
2013-07-02 06 53
2013-07-07 01 2091
2013-07-10 01 474
2013-07-10 02 279
2013-07-10 04 715
2013-07-10 12 157
2013-07-14 01 1081
2013-07-14 02 319
2013-07-17 01 1102
2013-07-17 02 541
2013-07-17 04 1184
2013-07-21 01 1325
2013-07-21 03 679
2013-07-25 02 3055
2013-07-28 01 1373
2013-07-28 02 513
2013-07-31 01 941
2013-07-31 02 1434
days when appliance is active : 14
no. of hours: 26
